In [ ]:
import os
def delete_all_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                delete_all_files_in_folder(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
import shutil, os, cv2
from tqdm.notebook import tqdm

# ARGS

target_path = "datasets/darwin_export_test_rotated"
to_clone = target_path
source_path_image = "SISTEMAS_UFF/Images/"
source_path_label = "SISTEMAS_UFF/Labels/output/"

os.makedirs(f"{target_path}/labels/val/",exist_ok=True)
os.makedirs(f"{target_path}/images/val/",exist_ok=True)
os.makedirs(f"{target_path}/labels/test/",exist_ok=True)
os.makedirs(f"{target_path}/images/test/",exist_ok=True)
os.makedirs(f"{target_path}/labels/train/",exist_ok=True)
os.makedirs(f"{target_path}/images/train/",exist_ok=True)

target_amount_train = 6
target_amount_val = 6
target_amount_test = 6

rotated_dataset = False

def enough_data_on_dataset(target_list_amount,current_list_amount)->bool:
    for i in range(len(target_list_amount)):
        if target_list_amount[i] > current_list_amount[i]:
            return False
    return True

def data_amount_required(target_list_amount)->bool:
    num = 0
    # check if there is a desired amount of data for the dataset
    for i in range(len(target_list_amount)):
        if target_list_amount[i] <= 1:
            num+=1
    # skip checks if there isnt any        
    if num == len(target_list_amount):
        return False
    
    # if there is a desired amount of any set, set all unespecified amounts to infinite
    for i in range(len(target_list_amount)):
        if target_list_amount[i] <= 1:
            # biggest number, use all available data in the csv
            target_list_amount[i] = float('inf')
    return True


target_amount_sets = [target_amount_train,target_amount_val,target_amount_test]

current_amount_train = 0
current_amount_val = 0
current_amount_test = 0
current_amount_sets = [current_amount_train,current_amount_val,current_amount_test]

not_skip_check = data_amount_required(target_amount_sets)

delete_all_files_in_folder(target_path)

number_file_total_img = 0
number_file_found_img = 0
files_not_found_img = []

number_file_total_labels = 0
number_file_found_labels = 0
files_not_found_labels = []

set_map =["train", "val", "test"]

header_read = False

#print(f"to_clone = {to_clone}")
#print(f"target_path = {target_path}")

folder_list = ["train","val","test"]

with open("SISTEMAS_UFF/dataset_rotated3.csv") as split_file:
    for line in tqdm(split_file):
        if header_read:
            img_id, img_id_float, img_name, label_name, set = line.strip().split(",")
            img_name=img_name.replace("Images/","")
            path_to_file_img = source_path_image+img_name
            path_to_file_labels = source_path_label+img_name.replace(".png",".txt")

            if not_skip_check:
                if target_amount_sets[int(set)-1] == current_amount_sets[int(set)-1]:
                    if enough_data_on_dataset(target_amount_sets,current_amount_sets):
                        break
                    continue
                
            # Ve se os arquivos existem nas pastas originais
            if os.path.exists(path_to_file_img):
                number_file_found_img += 1

                if os.path.exists(path_to_file_labels):
                    number_file_found_labels += 1

                    target_folder = set_map[int(set)-1]
                    target_img_path = target_path +"/images/"+target_folder+"/"+ img_name[:-4]
                    target_label_path = target_path +"/labels/"+target_folder+"/"+ img_name[:-4]

                    #copia o .txt com os labels para ás pastas target
                    shutil.copy(path_to_file_labels,target_label_path+".txt")
        
                    #se a face for NegY ou PosY tem q girar ela para ficar na mesma orientação que as labels
                    if target_img_path.lower().endswith("negy"):
                        img = cv2.imread(path_to_file_img)
                        if rotated_dataset:
                            img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
                        cv2.imwrite(target_img_path+".png",img)

                    elif target_img_path.lower().endswith("posy"):
                        img = cv2.imread(path_to_file_img)
                        if rotated_dataset:
                            img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
                        cv2.imwrite(target_img_path+".png",img)
                    
                    #se nao so copiar a imagem
                    else:
                        shutil.copy(path_to_file_img,target_img_path+".png")
                    
                    current_amount_sets[int(set)-1] += 1
                
                # separa o nome dos arquivos nao encontrados na pasta original
                else:
                    files_not_found_labels.append(path_to_file_labels)
                number_file_total_labels += 1
            else:
                files_not_found_img.append(path_to_file_img)
            number_file_total_img += 1

        if line.strip() == ",Unnamed: 0,Images,Labels,Set":
            header_read = True


print(f"Images found: {number_file_found_img}/{number_file_total_img}")
if number_file_found_img != number_file_total_img:
    for file in files_not_found_img:
        print(file)

print(f"Labels found: {number_file_found_labels}/{number_file_total_labels}")
if number_file_found_labels != number_file_total_labels:
    for file in files_not_found_labels:
        print(file)